## Initialize PySpark.

In [1]:
import os, sys, json, io
from pyspark.sql import *
from pyspark.sql.utils import StreamingQueryException
import sys
import json

os.environ['PYSPARK_PYTHON'] = '/usr/bin/python3'
os.environ['PYSPARK_DRIVER_PYTHON'] = '/usr/bin/python3'
sys.path.append('/class')

# Kafka variables
brokers = 'localhost:9092'
kafka_topic = 'stocks-json'
receiver_sleep_time = 4

# Connect to Spark 
if not 'sc' in locals():
    from initspark import initspark
    sc, spark, config = initspark(packages = ['kafka', 'kafka-sql', 'spark-avro'])



initializing pyspark
packages ['kafka', 'kafka-sql', 'spark-avro']
--packages org.apache.spark:spark-streaming-kafka-0-10_2.12:3.2.1,org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.1,org.apache.spark:spark-avro_2.12:3.2.1 pyspark-shell
:: loading settings :: url = jar:file:/usr/local/spark-3.2.1-bin-hadoop3.2/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.spark#spark-streaming-kafka-0-10_2.12 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.spark#spark-avro_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e185461d-4ed3-4d76-b75e-eeda53371b63;1.0
	confs: [default]
	found org.apache.spark#spark-streaming-kafka-0-10_2.12;3.2.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.2.1 in central
	found org.apache.kafka#kafka-clients;2.8.0 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.1 in central
	found org.spark-project.spark#unused;1.0.0 in spark-list
	found org.apache.hadoop#hadoop-client-api;3.3.1 in central
	found org.apache.htrace#htrace-core4;4.1.0-incubating i

pyspark initialized


## Basic batch source example

In [ ]:
! hadoop fs -rm -r /territories

In [ ]:
help(spark.read.csv)

In [ ]:
territories = spark.read.csv('file:///class/bigdata/1-apache-spark/territories.csv'
                             , header=True, inferSchema = True)
print(territories)
territories.show()
#territories.write.csv('hdfs://localhost:9000/territories', sep = '|')
#territories.write.json('hdfs://localhost:9000/territories_json')
#territories.write.parquet('hdfs://localhost:9000/territories_parquet')
                      
# territories.where('RegionID = 1').show()
# territories.groupby('RegionID').count().show()
# t2 = territories.where("TerritoryName like '%a%'")
# t3 = t2.groupby('RegionID').count()
# t4 = t3.filter('count > 5')

# (spark.read.csv('file:///class/2-apache-spark/territories.csv'
#                , header=True, inferSchema = True)
#       .where("TerritoryName like '%a%'")
#       .groupby('RegionID').count()
#       .filter('count > 5')
#       .show()
# )

#t4.show()
# territories.show()
territories.createOrReplaceTempView('territories')
spark.sql("""SELECT regionid, count(*) as cnt 
          from territories 
          where territoryname like '%a%' 
          group by regionid 
          order by cnt desc""").show()


## LAB: ## 
### The folder /class/datasets/northwind/ contains sample data in a variety of formats. CSV contains comma separated data without headers, CSVHeaders is the same data with headers. JSON, AVRO, ORC, PARQUET folders have the data in those formats. 
1. Read the CSVHeaders version of Categories into a DataFrame variable called categories. Print and show it to see what the data looks like.
2. Read the JSON version of Products into a DataFrame variable called products. Print and show it to see what the data looks like.
3. Using spark sql, turn each DataFrame variable into a temporary view and write a SQL statement to join the two into a new DataFrame variable that shows the ProductID, ProductName, CategoryID and CategoryName.
4. Using dot syntax take the joined DataFrame and count how many items are in each category.
5. Write the results to HDFS in a folder called /category_count
<p></p>

<details><summary>Click for <b>hint</b></summary>
<p>1. Use spark.read.csv and tell it to the file has headers and infer the schema. Use file:/// prefix to point to the files.</p>
    <p>2. Use spark.read.json</p>
    <p>3. Turn both DataFrames into a temporary view and write a standard SQL JOIN</p>
    <p>4. Use .grouby and .count</p>
    <p>5. Take the DataFrame and call .write.csv and save the results using hdfs:// prefix</p>
</details>

<details><summary>Click for <b>code</b></summary>
<p>

```python
categories = spark.read.csv('file:///class/datasets/northwind/CSVHeaders/categories', header = True, inferSchema = True)
products = spark.read.json('file:///class/datasets/northwind/JSON/products')
categories.createOrReplaceTempView('categories')
products.createOrReplaceTempView('products')
prod_cat = spark.sql("""SELECT c.CategoryID, c.CategoryName, p.ProductID, p.ProductName
FROM categories AS c
JOIN Products AS p ON c.CategoryID = p.CategoryID
""")
category_count = prod_cat.groupby('CategoryName').count()
category_count.show()
category_count.write.csv('hdfs://localhost:9000/category_count')
```
</p>
</details>

## Create a helper function to stream to a memory table.

In [2]:
def write_memory(df, queryname = 'debug', mode = "append"):
    # modes are: complete, update, append

    # if queryname in spark.catalog.listTables():
    #     spark.catalog.dropTempView(queryname)
    
    query = (df.writeStream 
            .format("memory")
            .queryName(queryname)
            .outputMode(mode)
            .start()
            )
    return query


## Define a streaming source and create a temp view to receive the results for debugging.

In [ ]:
kafka_topic = 'stocks-json'
df = (spark.readStream 
    .format("kafka") 
    .option("kafka.bootstrap.servers", brokers) 
    .option("subscribe", kafka_topic) 
    .option("startingOffsets", "earliest")
    .option("failOnDataLoss", False)
    .load()
    )

df.createOrReplaceTempView('table')
df1 = spark.sql("""SELECT CAST(value as string) as value, 'new data' as newfield from table""")

if 'debug1' in locals():
    debug1.stop()

#df1 = df.selectExpr("UPPER(CAST(value AS STRING)) as value")

debug1 = write_memory(df1, 'debug1')
#print(type(debug1))
#debug1.show()

## Query from the memory stream like it's a temporary view using `spark.sql`

In [ ]:
spark.sql("select * from debug1").take(10)

## You can stop and restart a memory stream whenever you like.

In [ ]:
debug1.stop()

In [ ]:
debug1 = write_memory(df1, 'debug1')

In [ ]:
spark.sql("select * from debug1").take(10)

## Spark SQL magic is also quite helpful.

In [3]:
%load_ext sparksql_magic
# pip install sparksql-magic

In [ ]:
%%sparksql
select * from debug1 order by value limit 10

## Stop a memory stream when you don't need it, as it can consume a lot of memory.

In [ ]:
debug1.stop()

## Let's try reading AVRO. First we are using schemaless AVRO messages so we need to read in a schema from a file or repository to apply to the message body to parse it into a structured format. This trick will take an AVRO schema file and turn it into a JSON string which can then be converted into a Spark struct object suitable for use in the deserializing process.

In [ ]:
stock_schema = '''{
    "namespace": "stock.avro",
    "type": "record",
    "name": "Stock",
    "fields": [
        {"name": "event_time", "type": "string"},
        {"name": "symbol",  "type": "string"},
        {"name": "price", "type": "float"},
        {"name": "quantity", "type": "int"}
    ]
}'''

stock_schema = open("stock.avsc", "r").read()
print('stock_schema', stock_schema)

stock_struct = spark.read.format("avro").option("avroSchema", stock_schema).load().schema
print('stock_struct', stock_struct)


## LAB: ## 
### Using the stocks-json example do the following to start up and read an AVRO stream instead:
1. Open a new terminal window and cd /class/1-producers-and-consumers
2. Run the 3-python-kafka-avro-producer.py to start making messages
3. Open another terminal window and cd /class//1-producers-and-consumers
4. Run 4-python-kafka-avro-consumer.py to show the messages are being created and sent.
5. In the cells below write Spark code based on the JSON example that can read the AVRO stream and simple display it. 

<p></p>

<details><summary>Click for <b>code</b></summary>
<p>

```python
brokers = 'localhost:9092'
kafka_topic = 'stocks-avro'
receiver_sleep_time = 4

df = (spark.readStream 
    .format("kafka") 
    .option("kafka.bootstrap.servers", brokers) 
    .option("subscribe", kafka_topic) 
    .option("startingOffsets", "earliest")
    .option("failOnDataLoss", False)
    .load()
    )
print('df', df)

if 'debug2' in locals():
    debug2.stop()
debug2 = write_memory(df, 'debug2')

# In a new cell
%%sparksql
select timestamp, key, value from debug2 order by timestamp desc limit 10    
```
</details>

In [ ]:
brokers = 'localhost:9092'
kafka_topic = 'stocks-avro'
receiver_sleep_time = 4

df = (spark.readStream 
    .format("kafka") 
    .option("kafka.bootstrap.servers", brokers) 
    .option("subscribe", kafka_topic) 
    .option("startingOffsets", "earliest")
    .option("failOnDataLoss", False)
    .load()
    )
print('df', df)

if 'debug2' in locals():
    debug2.stop()
debug2 = write_memory(df, 'debug2')


In [ ]:
%%sparksql
select timestamp, key, value from debug2 order by timestamp desc limit 10    

### Stop the memory stream object you created for the AVRO lab.

In [ ]:
debug2.stop()

## Next steps are to parse the message body and keep whatever metadata from the message we're interested in, and turn that into a DataFrame object we can work with and do whatever we want with the results. In this case in involves converting the key back to a UUID and the message body into Python dictionary.

In [ ]:
from pyspark.sql.functions import *
import uuid

def convert_uuid(value: bytes) -> str:
    # value is a bytearray in this case coming from spark
    ret = uuid.UUID(bytes = bytes(value))
    return str(ret)

convert_uuid_udf = udf(convert_uuid, StringType())
    
from pyspark.sql.avro.functions import from_avro, to_avro

# Could have read the schema from a file as shown earlier but just put it here so it's easier to see it.
stock_schema = """{
    "type": "record",
    "name": "Stock",
    "fields": [
        {"name": "event_time", "type": "string"},
        {"name": "symbol",  "type": "string"},
        {"name": "price", "type": "float"},
        {"name": "quantity", "type": "int"}
    ]
}"""

# Select the three columns we're interested in
# df3 = df.select("timestamp", "key", "value")

# Do some manipulating on the columns to make them into something meaningful
df3 = df.select("timestamp"
                , convert_uuid_udf(col("key")).alias("key")
                , from_avro(df.value, stock_schema, options = {"mode":"PERMISSIVE"}).alias("value"))

# We end up with three columns called timestamp, key and value, but value is a single column of the datatype
# struct, so this trick will flatten it out so we end up with six normal columns.
df3 = df3.select(*(df3.columns), col("value.*")).drop('value')

#df3 = df3.where("symbol = 'GOOG'")
df3.createOrReplaceTempView('stocks')

print('df3', df3)
if 'debug3' in locals():
    debug3.stop()
debug3 = write_memory(df3, 'debug3')

In [ ]:
%%sparksql
select * from debug3 order by timestamp desc

In [ ]:
debug3.stop()

## Here's the same thing for the JSON stream using the from_json function instead of from_avro.

In [4]:
from pyspark.sql.functions import *
import uuid

brokers = 'localhost:9092'
kafka_topic = 'stocks-json'
receiver_sleep_time = 4

stock_schema = open("stock.avsc", "r").read()
print('stock_schema', stock_schema)

stock_struct = spark.read.format("avro").option("avroSchema", stock_schema).load().schema
print('stock_struct', stock_struct)

df = (spark.readStream 
    .format("kafka") 
    .option("kafka.bootstrap.servers", brokers) 
    .option("subscribe", kafka_topic) 
    .option("startingOffsets", "earliest")
    .option("failOnDataLoss", False)
#    .option("kafka.group.id", "stock-json-spark-group")
    .load()
    )
print('df', df)


def convert_uuid(value):
    # value is a bytearray in this case coming from spark
    ret = uuid.UUID(bytes = bytes(value))
    return str(ret)

convert_uuid_udf = udf(convert_uuid, StringType())

# keep the key and timestamp and convert the value from bytes to string
#df1 = df.select(col("key"), "timestamp", expr("CAST(value AS STRING) as value"))
df1 = df.select(convert_uuid_udf(col("key")).alias("key"), "timestamp", expr("CAST(value AS STRING) as value"))
print('df1', df1)

# cast the string json to a struct
# keep all the columns we selected and convery the JSON string into a struct object and remove the string version
df2 = df1.select(*df1.columns, from_json(df1.value, stock_struct).alias("value2")).drop('value')
print('df2', df2)

# flatten the struct to a normal DataFrame
df4 = df2.select(*(df2.columns), col("value2.*")).drop('value2')
print('df4', df4)

if 'debug4' in locals():
    debug4.stop()
    
debug4 = write_memory(df4, 'debug4')


stock_schema {
    "namespace": "stock.avro",
    "type": "record",
    "name": "Stock",
    "fields": [
        {"name": "event_time", "type": "string"},
        {"name": "symbol",  "type": "string"},
        {"name": "price", "type": "float"},
        {"name": "quantity", "type": "int"}
    ]
}
stock_struct StructType(List(StructField(event_time,StringType,true),StructField(symbol,StringType,true),StructField(price,FloatType,true),StructField(quantity,IntegerType,true)))
df DataFrame[key: binary, value: binary, topic: string, partition: int, offset: bigint, timestamp: timestamp, timestampType: int]
df1 DataFrame[key: string, timestamp: timestamp, value: string]
df2 DataFrame[key: string, timestamp: timestamp, value2: struct<event_time:string,symbol:string,price:float,quantity:int>]
df4 DataFrame[key: string, timestamp: timestamp, event_time: string, symbol: string, price: float, quantity: int]


In [7]:
%%sparksql 
select * from debug4 order by event_time desc 

only showing top 20 row(s)


key,timestamp,event_time,symbol,price,quantity
c10a2a12-6df0-43f9-8225-6cd3148c1eeb,2022-03-24 16:19:36.152000,2022-03-24 16:19:36,GOOG,262.2300109863281,880
5934415c-4a5e-4e59-9d6d-7514a6d9739b,2022-03-24 16:19:36.151000,2022-03-24 16:19:36,AAPL,247.8000030517578,16
a3eec5fd-225e-45a8-9a6f-a40b17002dd6,2022-03-24 16:19:36.148000,2022-03-24 16:19:36,MSFT,152.5,391
0a218655-fafe-4819-9f6c-8654c083b8db,2022-03-24 16:19:32.148000,2022-03-24 16:19:32,AAPL,274.1600036621094,492
7c61f711-7b1c-4076-b012-5dd823808091,2022-03-24 16:19:32.148000,2022-03-24 16:19:32,GOOG,141.24000549316406,607
6cb7f41d-3bb4-402a-bf72-3b3e89ac04f1,2022-03-24 16:19:32.144000,2022-03-24 16:19:32,MSFT,164.16000366210938,170
3f2b48e4-90a5-4722-b9da-4d5e3a19ffbc,2022-03-24 16:19:28.143000,2022-03-24 16:19:28,AAPL,172.4199981689453,347
ef8a1ecc-bbdf-4257-b5b8-9a9bf2ab71e3,2022-03-24 16:19:28.144000,2022-03-24 16:19:28,GOOG,195.0399932861328,716
5d7175e9-711a-4489-a4b3-63873164d323,2022-03-24 16:19:28.139000,2022-03-24 16:19:28,MSFT,177.0,599
6b513cb5-2b50-4493-89a4-de228d4b41d8,2022-03-24 16:19:24.140000,2022-03-24 16:19:24,GOOG,166.6999969482422,124


In [8]:
debug4.stop()


## Now that we have a normal DataFrame, let's manipulate it how we want and write the results out to another stream. Try the following, it will fail. Read why.

In [ ]:
print(df4)
df4.createOrReplaceTempView('stocks2')
spark.sql('SELECT symbol, count(*) as cnt, sum(quantity) as qty from stocks2 group by symbol').show()

### Streaming sources can't be aggregated unless we add a window to them.

In [9]:
fixed_window = (df4.select("timestamp", "symbol", "quantity")
        .withWatermark("timestamp", "10 seconds") 
        .groupBy("symbol", window("timestamp", "10 seconds").alias("window")) 
        .agg(sum("quantity").alias("sum"))
        )
print(fixed_window)

if 'debug5' in locals():
    debug5.stop()
debug5 = write_memory(fixed_window, 'debug5')


DataFrame[symbol: string, window: struct<start:timestamp,end:timestamp>, sum: bigint]


## We can see that we get aggregate by symbol every ten seconds. This data can be written off somewhere like a SQL or NoSQL database or forwarded as a new message to create a streaming aggregation.

In [10]:
%%sparksql
select * from debug5 order by window desc, symbol limit 9

symbol,window,sum


In [ ]:
debug5.stop()

## Sliding windows are similar except you give it two parameters, the first is the total length of the window and the second is the refresh interval. In this case, the windows will overlap.

In [ ]:
sliding_window = (df4.select("timestamp", "symbol","quantity")
        .withWatermark("timestamp", "10 seconds") 
        .groupBy(window("timestamp", "30 seconds", "10 seconds").alias("window"), "symbol") 
        .agg(sum("quantity").alias("sum"))
        )
print(sliding_window)

debug6 = write_memory(sliding_window, 'debug6')


In [ ]:
%%sparksql
select * from debug6 order by window desc, symbol limit 21


In [ ]:
debug6.stop()

## Session Window is similar but used to group data that represents a continuous stream of activity. The time specifies a timeout period or period of inactivity that indicates when a session should end.

In [ ]:
session_window = (df4.select("timestamp", "symbol","quantity")
        .withWatermark("timestamp", "10 seconds") 
        .groupBy(session_window("timestamp", "5 minutes").alias("window"), "symbol") 
        .agg(sum("quantity").alias("sum"))
        )
print(session_window)

debug7 = write_memory(session_window, 'debug7')


## Let's join the streaming aggregation with a static reference table.

In [ ]:
x = sc.parallelize([('AAPL', 'Apple'), ('MSFT', 'Microsoft'), ('GOOG','Google')])
stocks = spark.createDataFrame(x, 'symbol:string, name:string')
stocks.createOrReplaceTempView('stocks')
fixed_window.createOrReplaceTempView('trades')

joined_aggregate = spark.sql("""
SELECT t.*, s.name
FROM trades as t
JOIN stocks as s on t.symbol = s.symbol
""")

debug8 = write_memory(joined_aggregate, 'debug8')



In [ ]:
%%sparksql
select * from debug8 order by window desc, symbol limit 9


In [ ]:
debug8.stop()

### Sample of what a consume record in AVRO looks like

ConsumerRecord(topic='stocks-avro', partition=0, offset=40679, timestamp=1645243362535, timestamp_type=0, key=b'\xd6\x0cgMs<By\xb8\xcaR\x02\xe0\xfa\x93\x14', value=b'Obj\x01\x04\x14avro.codec\x08null\x16avro.schema\xc6\x03{"type": "record", "name": "Stock", "namespace": "stock.avro", "fields": [{"type": "string", "name": "event_time"}, {"type": "string", "name": "symbol"}, {"type": "float", "name": "price"}, {"type": "int", "name": "quantity"}]}\x00$\x7f#w\xeaD\xdchK\xc5!\xf85\x10\xc8J\x02>&2022-02-19 04:02:42\x08MSFT\xa4p\xdfB\xf0\t$\x7f#w\xeaD\xdchK\xc5!\xf85\x10\xc8J', headers=[], checksum=None, serialized_key_size=16, serialized_value_size=328, serialized_header_size=-1)
ConsumerRecord(topic='stocks-avro', partition=0, offset=40679, timestamp=1645243362535, timestamp_type=0, key=b'\xd6\x0cgMs<By\xb8\xcaR\x02\xe0\xfa\x93\x14', value=b'Obj\x01\x04\x14avro.codec\x08null\x16avro.schema\xc6\x03{"type": "record", "name": "Stock", "namespace": "stock.avro", "fields": [{"type": "string", "name": "event_time"}, {"type": "string", "name": "symbol"}, {"type": "float", "name": "price"}, {"type": "int", "name": "quantity"}]}\x00$\x7f#w\xeaD\xdchK\xc5!\xf85\x10\xc8J\x02>&2022-02-19 04:02:42\x08MSFT\xa4p\xdfB\xf0\t$\x7f#w\xeaD\xdchK\xc5!\xf85\x10\xc8J', headers=[], checksum=None, serialized_key_size=16, serialized_value_size=328, serialized_header_size=-1)

ConsumerRecord(topic='stocks-avro', partition=0, offset=40814, timestamp=1645243470645, timestamp_type=0, key=b'\xc8\xeb\xc2\xe9O\xaaJ\x86\x83\x85\xb9\xd7\xf46\xea\x8f', value=b'Obj\x01\x04\x14avro.codec\x08null\x16avro.schema\xc6\x03{"type": "record", "name": "Stock", "namespace": "stock.avro", "fields": [{"type": "string", "name": "event_time"}, {"type": "string", "name": "symbol"}, {"type": "float", "name": "price"}, {"type": "int", "name": "quantity"}]}\x00\xe8\xf5x\r\xbf\x8aC\x98&\xaf\x13iz\x9dp\x13\x02>&2022-02-19 04:04:30\x08MSFT\xc3\xf5\x11C\xac\x04\xe8\xf5x\r\xbf\x8aC\x98&\xaf\x13iz\x9dp\x13', headers=[], checksum=None, serialized_key_size=16, serialized_value_size=328, serialized_header_size=-1)
